# 第四節 – SLM 與 LLM 比較

比較小型語言模型（SLM）與透過 Foundry Local 運行的大型語言模型（LLM）之間的延遲和樣本回應品質。


## ⚡ 快速開始

**記憶體優化設置（更新版）：**
1. 模型自動選擇 CPU 版本（適用於任何硬體）
2. 使用 `qwen2.5-3b` 替代 7B（節省約 4GB RAM）
3. 端口自動檢測（無需手動配置）
4. 所需總 RAM：建議約 8GB（模型 + 作業系統）

**終端設置（30秒）：**
```bash
foundry service start
foundry model run phi-4-mini
foundry model run qwen2.5-3b
```

然後運行此筆記本！ 🚀


### 說明：依賴項安裝
安裝執行計時和聊天請求所需的最少套件（`foundry-local-sdk`、`openai`、`numpy`）。可以安全地重複執行，具有冪等性。


# 情境
比較一個代表性的小型語言模型（SLM）與較大型模型在單一提示上的表現，以說明以下取捨：
- **延遲差異**（實際時間，秒）
- **Token 使用量**（若可用）作為吞吐量的代理指標
- **樣本質量輸出**，便於快速檢視
- **加速計算**，量化性能提升

**環境變數：**
- `SLM_ALIAS` - 小型語言模型（預設：phi-4-mini，約 4GB RAM）
- `LLM_ALIAS` - 較大型語言模型（預設：qwen2.5-7b，約 7GB RAM）
- `COMPARE_PROMPT` - 用於比較的測試提示
- `COMPARE_RETRIES` - 為提高穩定性進行的重試次數（預設：2）
- `FOUNDRY_LOCAL_ENDPOINT` - 覆蓋服務端點（若未設定則自動檢測）

**運作方式（官方 SDK 模式）：**
1. **FoundryLocalManager** 初始化並管理 Foundry Local 服務
2. 若服務未運行，將自動啟動（無需手動設置）
3. 模型從別名自動解析為具體 ID
4. 選擇硬體優化版本（CUDA、NPU 或 CPU）
5. 使用 OpenAI 兼容的客戶端執行聊天完成
6. 捕捉指標：延遲、Token 使用量、輸出質量
7. 比較結果以計算加速比

此微型比較有助於判斷在您的使用案例中，何時需要路由至較大型模型。

**SDK 參考：** 
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- Workshop Utils: 使用 ../samples/workshop_utils.py 中的官方模式

**主要優勢：**
- ✅ 自動服務發現與初始化
- ✅ 若服務未運行，則自動啟動
- ✅ 內建模型解析與快取
- ✅ 硬體優化（CUDA/NPU/CPU）
- ✅ OpenAI SDK 兼容性
- ✅ 強大的錯誤處理與重試機制
- ✅ 本地推理（無需雲端 API）


## 🚨 先決條件：Foundry Local 必須正在運行！

**在執行此筆記本之前**，請確保已設置 Foundry Local 服務：

### 快速啟動命令（在終端中執行）：

```bash
# 1. Start the Foundry Local service
foundry service start

# 2. Load the default models used in this comparison (CPU-optimized)
foundry model run phi-4-mini
foundry model run qwen2.5-3b

# 3. Verify models are loaded
foundry model ls

# 4. Check service health
foundry service status
```

### 替代模型（如果默認模型不可用）：

```bash
# Even smaller alternatives (if memory is very limited)
foundry model run phi-3.5-mini
foundry model run qwen2.5-0.5b

# Or update the environment variables in this notebook:
# SLM_ALIAS = 'phi-3.5-mini'
# LLM_ALIAS = 'qwen2.5-1.5b'  # Or qwen2.5-0.5b for minimum memory
```

⚠️ **如果跳過這些步驟**，在執行以下筆記本單元格時，您將看到 `APIConnectionError`。


In [29]:
# Install dependencies
!pip install -q foundry-local-sdk openai numpy requests

### 說明：核心匯入
引入計時工具以及 Foundry Local / OpenAI 客戶端，用於獲取模型資訊並執行聊天完成。


In [30]:
import os, time, json
from foundry_local import FoundryLocalManager
from openai import OpenAI
import sys
sys.path.append('../samples')
from workshop_utils import get_client, chat_once

### 說明：別名與提示設置
定義可根據環境配置的別名，用於小型模型與大型模型，以及比較提示。調整環境變數以嘗試不同的模型系列或任務。


In [31]:
# Default to CPU models for better memory efficiency
SLM = os.getenv('SLM_ALIAS', 'phi-4-mini')  # Auto-selects CPU variant
LLM = os.getenv('LLM_ALIAS', 'qwen2.5-3b')  # Smaller LLM, more memory-friendly
PROMPT = os.getenv('COMPARE_PROMPT', 'List 5 benefits of local AI inference.')
# Endpoint is now managed by FoundryLocalManager - it auto-detects or can be overridden
ENDPOINT = os.getenv('FOUNDRY_LOCAL_ENDPOINT', None)

### 💡 記憶體優化配置

**此筆記本預設使用記憶體高效模型：**
- `phi-4-mini` → 約 4GB RAM（Foundry Local 自動選擇 CPU 版本）
- `qwen2.5-3b` → 約 3GB RAM（取代需要約 7GB+ 的 7B）

**埠自動檢測：**
- Foundry Local 可能使用不同的埠（通常是 55769 或 59959）
- 下方的診斷單元會自動檢測正確的埠
- 無需手動配置！

**如果您的記憶體有限（<8GB），請使用更小的模型：**
```python
SLM = 'phi-3.5-mini'      # ~2GB
LLM = 'qwen2.5-0.5b'      # ~500MB
```


In [32]:
# Display current configuration
print("="*60)
print("CURRENT CONFIGURATION")
print("="*60)
print(f"SLM Model:     {SLM}")
print(f"LLM Model:     {LLM}")
print(f"SDK Pattern:   FoundryLocalManager (official)")
print(f"Endpoint:      {ENDPOINT or 'Auto-detect'}")
print(f"Test Prompt:   {PROMPT[:50]}...")
print(f"Retry Count:   2")
print("="*60)
print("\n💡 Using official Foundry SDK pattern from workshop_utils")
print("   → FoundryLocalManager handles service lifecycle")
print("   → Automatic model resolution and hardware optimization")
print("   → OpenAI-compatible API for inference")

CURRENT CONFIGURATION
SLM Model:     phi-4-mini
LLM Model:     qwen2.5-7b
SDK Pattern:   FoundryLocalManager (official)
Endpoint:      Auto-detect
Test Prompt:   List 5 benefits of local AI inference....
Retry Count:   2

💡 Using official Foundry SDK pattern from workshop_utils
   → FoundryLocalManager handles service lifecycle
   → Automatic model resolution and hardware optimization
   → OpenAI-compatible API for inference


### 說明：執行輔助工具（Foundry SDK 模式）
使用官方 Foundry Local SDK 模式，根據 Workshop 範例中的文件進行：

**方法：**
- **FoundryLocalManager** - 初始化並管理 Foundry Local 服務
- **自動檢測** - 自動發現端點並處理服務生命週期
- **模型解析** - 將別名解析為完整的模型 ID（例如，phi-4-mini → phi-4-mini-instruct-cpu）
- **硬體優化** - 為可用硬體選擇最佳變體（CUDA、NPU 或 CPU）
- **OpenAI 客戶端** - 配置為使用管理器的端點以訪問與 OpenAI 兼容的 API

**韌性功能：**
- 指數回退重試邏輯（可通過環境進行配置）
- 如果服務未運行，則自動啟動服務
- 初始化後的連接驗證
- 優雅的錯誤處理，提供詳細的錯誤報告
- 模型緩存以避免重複初始化

**結果結構：**
- 延遲測量（實際時間）
- Token 使用追蹤（如果可用）
- 範例輸出（為了可讀性已截短）
- 失敗請求的錯誤詳細信息

此模式利用了 workshop_utils 模組，該模組遵循官方 SDK 模式。

**SDK 參考：**
- 主倉庫：https://github.com/microsoft/Foundry-Local
- Python SDK：https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- Workshop Utils：../samples/workshop_utils.py


In [39]:
def setup(alias: str, endpoint: str = None, retries: int = 3):
    """
    Initialize a Foundry Local model connection using official SDK pattern.
    
    This follows the workshop_utils pattern which uses FoundryLocalManager
    to properly initialize the Foundry Local service and resolve models.
    
    Args:
        alias: Model alias (e.g., 'phi-4-mini', 'qwen2.5-3b')
        endpoint: Optional endpoint override (usually auto-detected)
        retries: Number of connection attempts (default: 3)
    
    Returns:
        tuple: (manager, client, model_id, metadata) or (None, None, alias, error_metadata) if failed
    """
    import time
    
    last_err = None
    current_delay = 2  # seconds
    
    for attempt in range(1, retries + 1):
        try:
            print(f"[Init] Connecting to '{alias}' (attempt {attempt}/{retries})...")
            
            # Use the workshop utility which follows the official SDK pattern
            manager, client, model_id = get_client(alias, endpoint=endpoint)
            
            print(f"[OK] Connected to '{alias}' -> {model_id}")
            print(f"     Endpoint: {manager.endpoint}")
            
            return manager, client, model_id, {
                'endpoint': manager.endpoint,
                'resolved': model_id,
                'attempts': attempt,
                'status': 'success'
            }
            
        except Exception as e:
            last_err = e
            error_msg = str(e)
            
            # Provide helpful error messages
            if "Connection error" in error_msg or "connection refused" in error_msg.lower():
                print(f"[ERROR] Cannot connect to Foundry Local service")
                print(f"        → Is the service running? Try: foundry service start")
                print(f"        → Is the model loaded? Try: foundry model run {alias}")
            elif "not found" in error_msg.lower():
                print(f"[ERROR] Model '{alias}' not found in catalog")
                print(f"        → Available models: Run 'foundry model ls' in terminal")
                print(f"        → Download model: Run 'foundry model download {alias}'")
            else:
                print(f"[ERROR] Setup failed: {type(e).__name__}: {error_msg}")
            
            if attempt < retries:
                print(f"[Retry] Waiting {current_delay:.1f}s before retry...")
                time.sleep(current_delay)
                current_delay *= 2  # Exponential backoff
    
    # All retries failed - provide actionable guidance
    print(f"\n❌ Failed to initialize '{alias}' after {retries} attempts")
    print(f"   Last error: {type(last_err).__name__}: {str(last_err)}")
    print(f"\n💡 Troubleshooting steps:")
    print(f"   1. Ensure Foundry Local service is running:")
    print(f"      → foundry service status")
    print(f"      → foundry service start (if not running)")
    print(f"   2. Ensure model is loaded:")
    print(f"      → foundry model run {alias}")
    print(f"   3. Check available models:")
    print(f"      → foundry model ls")
    print(f"   4. Try alternative models if '{alias}' isn't available")
    
    return None, None, alias, {
        'error': f"{type(last_err).__name__}: {str(last_err)}",
        'endpoint': endpoint or 'auto-detect',
        'attempts': retries,
        'status': 'failed'
    }


def run(client, model_id: str, prompt: str, max_tokens: int = 180, temperature: float = 0.5):
    """
    Run inference with the configured model using OpenAI SDK.
    
    Args:
        client: OpenAI client instance (configured for Foundry Local)
        model_id: Model identifier (resolved from alias)
        prompt: Input prompt
        max_tokens: Maximum response tokens (default: 180)
        temperature: Sampling temperature (default: 0.5)
    
    Returns:
        dict: Response with timing, tokens, and content
    """
    import time
    
    start = time.time()
    
    try:
        response = client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature
        )
        
        elapsed = time.time() - start
        
        # Extract response details
        content = response.choices[0].message.content
        
        # Try to extract token usage from multiple possible locations
        usage_info = {}
        if hasattr(response, 'usage') and response.usage:
            usage_info['prompt_tokens'] = getattr(response.usage, 'prompt_tokens', None)
            usage_info['completion_tokens'] = getattr(response.usage, 'completion_tokens', None)
            usage_info['total_tokens'] = getattr(response.usage, 'total_tokens', None)
        
        # Calculate approximate token count if API doesn't provide it
        # Rough estimate: ~4 characters per token for English text
        if not usage_info.get('total_tokens'):
            estimated_prompt_tokens = len(prompt) // 4
            estimated_completion_tokens = len(content) // 4
            estimated_total = estimated_prompt_tokens + estimated_completion_tokens
            usage_info['estimated_tokens'] = estimated_total
            usage_info['estimated_prompt_tokens'] = estimated_prompt_tokens
            usage_info['estimated_completion_tokens'] = estimated_completion_tokens
        
        return {
            'status': 'success',
            'content': content,
            'elapsed_sec': elapsed,
            'tokens': usage_info.get('total_tokens') or usage_info.get('estimated_tokens'),
            'usage': usage_info,
            'model': model_id
        }
        
    except Exception as e:
        elapsed = time.time() - start
        return {
            'status': 'error',
            'error': f"{type(e).__name__}: {str(e)}",
            'elapsed_sec': elapsed,
            'model': model_id
        }


print("✅ Execution helpers defined: setup(), run()")
print("   → Uses workshop_utils for proper SDK integration")
print("   → setup() initializes with FoundryLocalManager")
print("   → run() executes inference via OpenAI-compatible API")
print("   → Token counting: Uses API data or estimates if unavailable")

✅ Execution helpers defined: setup(), run()
   → Uses workshop_utils for proper SDK integration
   → setup() initializes with FoundryLocalManager
   → run() executes inference via OpenAI-compatible API
   → Token counting: Uses API data or estimates if unavailable


### 說明：預檢自測
使用 FoundryLocalManager 為兩個模型執行輕量級的連接性檢查。此檢查可驗證：
- 服務是否可訪問
- 模型是否能初始化
- 別名是否解析為實際的模型 ID
- 在執行比較之前，連接是否穩定

setup() 函數使用來自 workshop_utils 的官方 SDK 模式。


In [34]:
# Simplified diagnostic: Just verify service is accessible
import requests

def check_foundry_service():
    """Quick diagnostic to verify Foundry Local is running."""
    # Try common ports
    endpoints_to_try = [
        "http://localhost:59959",
        "http://127.0.0.1:59959", 
        "http://localhost:55769",
        "http://127.0.0.1:55769",
    ]
    
    print("[Diagnostic] Checking Foundry Local service...")
    
    for endpoint in endpoints_to_try:
        try:
            response = requests.get(f"{endpoint}/health", timeout=2)
            if response.status_code == 200:
                print(f"✅ Service is running at {endpoint}")
                
                # Try to list models
                try:
                    models_response = requests.get(f"{endpoint}/v1/models", timeout=2)
                    if models_response.status_code == 200:
                        models_data = models_response.json()
                        model_count = len(models_data.get('data', []))
                        print(f"✅ Found {model_count} models available")
                        if model_count > 0:
                            print("   Models:", [m.get('id', 'unknown') for m in models_data.get('data', [])[:5]])
                except Exception as e:
                    print(f"⚠️  Could not list models: {e}")
                
                return endpoint
        except requests.exceptions.ConnectionError:
            continue
        except Exception as e:
            print(f"⚠️  Error checking {endpoint}: {e}")
    
    print("\n❌ Foundry Local service not found!")
    print("\n💡 To fix this:")
    print("   1. Open a terminal")
    print("   2. Run: foundry service start")
    print("   3. Run: foundry model run phi-4-mini")
    print("   4. Run: foundry model run qwen2.5-3b")
    print("   5. Re-run this notebook")
    return None

# Run diagnostic
discovered_endpoint = check_foundry_service()

if discovered_endpoint:
    print(f"\n✅ Service detected (will be managed by FoundryLocalManager)")
else:
    print(f"\n⚠️  No service detected - FoundryLocalManager will attempt to start it")

[Diagnostic] Checking Foundry Local service...

❌ Foundry Local service not found!

💡 To fix this:
   1. Open a terminal
   2. Run: foundry service start
   3. Run: foundry model run phi-4-mini
   4. Run: foundry model run qwen2.5-3b
   5. Re-run this notebook

⚠️  No service detected - FoundryLocalManager will attempt to start it


In [35]:
# Quick Fix: Start service and load models from notebook
# Uncomment the commands you need:

# !foundry service start
# !foundry model run phi-4-mini
# !foundry model run qwen2.5-3b
# !foundry model ls

print("⚠️  The commands above are commented out.")
print("Uncomment them if you want to start the service from the notebook.")
print("")
print("💡 Recommended: Run these commands in a separate terminal instead:")
print("   foundry service start")
print("   foundry model run phi-4-mini")
print("   foundry model run qwen2.5-3b")

⚠️  The commands above are commented out.
Uncomment them if you want to start the service from the notebook.

💡 Recommended: Run these commands in a separate terminal instead:
   foundry service start
   foundry model run phi-4-mini
   foundry model run qwen2.5-3b


### 🛠️ 快速修復：從筆記本啟動 Foundry Local（可選）

如果上述診斷顯示服務未運行，您可以嘗試從這裡啟動：

**注意：** 這在 Windows 上效果最佳。在其他平台上，請使用終端命令。


### ⚠️ 排除連線錯誤

如果您看到 `APIConnectionError`，可能是 Foundry Local 服務未啟動或模型未載入。請嘗試以下步驟：

**1. 檢查服務狀態：**
```bash
# In a terminal (not in notebook):
foundry service status
```

**2. 啟動服務（如果未運行）：**
```bash
foundry service start
```

**3. 載入所需模型：**
```bash
# Load the models needed for comparison
foundry model run phi-4-mini
foundry model run qwen2.5-7b

# Or use alternative models:
foundry model run phi-3.5-mini
foundry model run qwen2.5-3b
```

**4. 驗證模型是否可用：**
```bash
foundry model ls
```

**常見問題：**
- ❌ 服務未運行 → 執行 `foundry service start`
- ❌ 模型未載入 → 執行 `foundry model run <model-name>`
- ❌ 埠衝突 → 檢查是否有其他服務正在使用該埠
- ❌ 防火牆阻擋 → 確保允許本地連線

**快速修復：** 在進行預檢查之前，執行以下診斷單元格。


In [36]:
preflight = {}
retries = 2  # Number of retry attempts

for a in (SLM, LLM):
    mgr, c, mid, info = setup(a, endpoint=ENDPOINT, retries=retries)
    # Keep the original status from info (either 'success' or 'failed')
    preflight[a] = info

print('\n[Pre-flight Check]')
for alias, details in preflight.items():
    status_icon = '✅' if details['status'] == 'success' else '❌'
    print(f"  {status_icon} {alias}: {details['status']} - {details.get('resolved', details.get('error', 'unknown'))}")

preflight

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1

[Pre-flight Check]
  ✅ phi-4-mini: success - Phi-4-mini-instruct-cuda-gpu:4
  ✅ qwen2.5-7b: success - qwen2.5-7b-instruct-cuda-gpu:3


{'phi-4-mini': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'Phi-4-mini-instruct-cuda-gpu:4',
  'attempts': 1,
  'status': 'success'},
 'qwen2.5-7b': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'qwen2.5-7b-instruct-cuda-gpu:3',
  'attempts': 1,
  'status': 'success'}}

### ✅ 預檢：模型可用性

此單元格在執行比較之前，會驗證兩個模型是否可以在配置的端點上被訪問。


### 說明：執行比較並收集結果
使用官方 Foundry SDK 模式迭代兩個別名：
1. 使用 setup() 初始化每個模型（使用 FoundryLocalManager）
2. 使用與 OpenAI 相容的 API 執行推論
3. 捕捉延遲、tokens 和範例輸出
4. 生成包含比較分析的 JSON 摘要

這與 session04/model_compare.py 中的 Workshop 範例遵循相同的模式。


In [40]:
results = []
retries = 2  # Number of retry attempts

for alias in (SLM, LLM):
    mgr, client, mid, info = setup(alias, endpoint=ENDPOINT, retries=retries)
    if client:
        r = run(client, mid, PROMPT)
        results.append({'alias': alias, **r})
    else:
        # If setup failed, record error
        results.append({
            'alias': alias,
            'status': 'error',
            'error': info.get('error', 'Setup failed'),
            'elapsed_sec': 0,
            'tokens': None,
            'model': alias
        })

# Display results
print(json.dumps(results, indent=2))

# Quick comparative view
print('\n' + '='*80)
print('COMPARISON SUMMARY')
print('='*80)
print(f"{'Alias':<20} {'Status':<15} {'Latency(s)':<15} {'Tokens':<15}")
print('-'*80)

for row in results:
    status = row.get('status', 'unknown')
    status_icon = '✅' if status == 'success' else '❌'
    latency_str = f"{row.get('elapsed_sec', 0):.3f}" if row.get('elapsed_sec') else 'N/A'
    
    # Handle token display - show if available or indicate estimated
    tokens = row.get('tokens')
    usage = row.get('usage', {})
    if tokens:
        if 'estimated_tokens' in usage:
            tokens_str = f"~{tokens} (est.)"
        else:
            tokens_str = str(tokens)
    else:
        tokens_str = 'N/A'
    
    print(f"{status_icon} {row['alias']:<18} {status:<15} {latency_str:<15} {tokens_str:<15}")

print('-'*80)

# Show detailed token breakdown if available
print("\nDetailed Token Usage:")
for row in results:
    if row.get('status') == 'success' and row.get('usage'):
        usage = row['usage']
        print(f"\n  {row['alias']}:")
        if 'prompt_tokens' in usage and usage['prompt_tokens']:
            print(f"    Prompt tokens:     {usage['prompt_tokens']}")
            print(f"    Completion tokens: {usage['completion_tokens']}")
            print(f"    Total tokens:      {usage['total_tokens']}")
        elif 'estimated_tokens' in usage:
            print(f"    Estimated prompt:     {usage['estimated_prompt_tokens']}")
            print(f"    Estimated completion: {usage['estimated_completion_tokens']}")
            print(f"    Estimated total:      {usage['estimated_tokens']}")
            print(f"    (API did not provide token counts - using ~4 chars/token estimate)")

print('\n' + '='*80)

# Calculate speedup if both succeeded
if len(results) == 2 and all(r.get('status') == 'success' and r.get('elapsed_sec') for r in results):
    speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec']
    print(f"\n💡 SLM is {speedup:.2f}x faster than LLM for this prompt")
    
    # Compare token throughput if available
    slm_tokens = results[0].get('tokens', 0)
    llm_tokens = results[1].get('tokens', 0)
    if slm_tokens and llm_tokens:
        slm_tps = slm_tokens / results[0]['elapsed_sec']
        llm_tps = llm_tokens / results[1]['elapsed_sec']
        print(f"   SLM throughput: {slm_tps:.1f} tokens/sec")
        print(f"   LLM throughput: {llm_tps:.1f} tokens/sec")
        
elif any(r.get('status') == 'error' for r in results):
    print(f"\n⚠️  Some models failed - check errors above")
    print("   Ensure Foundry Local is running: foundry service start")
    print("   Ensure models are loaded: foundry model run <model-name>")

results

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[
  {
    "alias": "phi-4-mini",
    "status": "success",
    "content": "1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the n

[{'alias': 'phi-4-mini',
  'status': 'success',
  'content': '1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the need for data transmission over the network, which can save bandwidth and reduce the risk of network congestion.\n\n4. Improved Reliability: Local processing can be more reliable, as it is less dependent on network connectivity. This is particularly important in scenarios where network connectivity is unreliable or intermittent.\n\n5. Scalability: Local AI inference can be easily scaled by adding more local processing units, making it easier to handle increasing data volumes or m

### 解讀結果

**關鍵指標：**
- **延遲**：越低越好 - 表示回應時間更快
- **Tokens**：吞吐量越高 = 處理的 tokens 越多
- **路由**：確認使用了哪個 API 端點

**何時使用 SLM 與 LLM：**
- **SLM（小型語言模型）**：回應速度快，資源使用較少，適合簡單任務
- **LLM（大型語言模型）**：品質更高，推理能力更強，適用於對品質要求較高的情況

**下一步：**
1. 嘗試不同的提示，觀察複雜性如何影響比較結果
2. 試驗其他模型組合
3. 使用 Workshop 的路由範例（Session 06），根據任務複雜度智能路由


In [38]:
# Final Validation Check
print("="*70)
print("VALIDATION SUMMARY")
print("="*70)
print(f"✅ SLM Model: {SLM}")
print(f"✅ LLM Model: {LLM}")
print(f"✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager")
print(f"✅ Pre-flight passed: {all(v['status'] == 'success' for v in preflight.values()) if 'preflight' in dir() else 'Not run yet'}")
print(f"✅ Comparison completed: {len(results) == 2 if 'results' in dir() else 'Not run yet'}")
print(f"✅ Both models responded: {all(r.get('status') == 'success' for r in results) if 'results' in dir() and results else 'Not run yet'}")
print("="*70)

# Check for common configuration issues
issues = []
if 'LLM' in dir() and LLM not in ['qwen2.5-3b', 'qwen2.5-0.5b', 'qwen2.5-1.5b', 'qwen2.5-7b', 'phi-3.5-mini']:
    issues.append(f"⚠️  LLM is '{LLM}' - expected qwen2.5-3b for memory efficiency")
if 'preflight' in dir() and not all(v['status'] == 'success' for v in preflight.values()):
    issues.append("⚠️  Pre-flight check failed - models not accessible")
if 'results' in dir() and results and not all(r.get('status') == 'success' for r in results):
    issues.append("⚠️  Comparison incomplete - check for errors above")

if not issues and 'results' in dir() and results and all(r.get('status') == 'success' for r in results):
    print("🎉 ALL CHECKS PASSED! Notebook completed successfully.")
    print(f"   SLM ({SLM}) vs LLM ({LLM}) comparison completed.")
    if len(results) == 2:
        speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec'] if results[0]['elapsed_sec'] > 0 else 0
        print(f"   Performance: SLM is {speedup:.2f}x faster")
elif issues:
    print("\n⚠️  Issues detected:")
    for issue in issues:
        print(f"   {issue}")
    print("\n💡 Troubleshooting:")
    print("   1. Ensure service is running: foundry service start")
    print("   2. Load models: foundry model run phi-4-mini && foundry model run qwen2.5-7b")
    print("   3. Check model list: foundry model ls")
else:
    print("\n💡 Run all cells above first, then re-run this validation.")
print("="*70)

VALIDATION SUMMARY
✅ SLM Model: phi-4-mini
✅ LLM Model: qwen2.5-7b
✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager
✅ Pre-flight passed: True
✅ Comparison completed: True
✅ Both models responded: True
🎉 ALL CHECKS PASSED! Notebook completed successfully.
   SLM (phi-4-mini) vs LLM (qwen2.5-7b) comparison completed.
   Performance: SLM is 5.14x faster



---

**免責聲明**：  
本文件已使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們努力確保翻譯的準確性，但請注意，自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應被視為權威來源。對於關鍵信息，建議使用專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或錯誤解釋不承擔責任。
